In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib qt

In [2]:
# Parameters definition

c = 3e8
pi = np.pi
f = 1.0e10

t = np.linspace(0, 100/f, 100)
exp = np.exp(1j * 2 * pi * f * t)

e0 = 8.85e-12
mu0 = 4*pi*1e-7
l0 = 15e-2
n0 = 1
eta0 = 377
z0 = np.linspace(-l0, 0, 1500)

n1 = 3
e1 = n1**2 * e0
l1 = 10e-2 
z1 = np.linspace(0, l1, 1000)
eta1 = eta0 / n1

n2 = 1.5
e2 = n2**2 * e0
l2 = 5e-2
z2 = np.linspace(0, l2, 500)
eta2 = eta0 / n2

n3 = 1
e3 = n3**2 * e0
l3 = 15e-2
z3 = np.linspace(0, l3, 1500)
eta3 = eta0 / n3

In [3]:
def propagation(E0, H0, n, z, eta):
    k = 2 * pi * f * n / c
    print(k)
    E = E0 * np.cos(k*z) - H0 * 1j * eta * np.sin(k*z)
    H = - E0 * 1j * np.sin(k*z) / eta + H0 * np.cos(k*z)
    return E, H 

In [4]:
# Recursive computation of the reflection coefficients.
# p at the end of variables name indicates primus (value to the right of the interface)

Gamma3 = (n2-n3)/(n2+n3)
Gamma2p = Gamma3 * np.exp(-1j*2*pi*f*n2*l2/c)
rho2 = (n1-n2)/(n1+n2)
Gamma2 = (rho2 + Gamma2p) / (1 + rho2*Gamma2p)
Gamma1p = Gamma2 * np.exp(-1j*2*pi*f*n1*l1/c)
rho1 = (n0-n1)/(n0+n1)
Gamma1 = (rho1 + Gamma1p) / (1 + rho1*Gamma1p)

In [5]:
Ef = 1                                          # E+
Eb = Gamma1                                     # E-

E0 = (Ef + Eb)                               # Actual E field
H0 = (Ef - Eb) / eta0                       # Actual H field

E0, H0 = propagation(E0, H0, n0, z0, eta0)

E1, H1 = propagation(E0[-1], H0[-1], n1, z1, eta1)

E2, H2 = propagation(E1[-1], H1[-1], n2, z2, eta2)

E3, H3 = propagation(E2[-1], H2[-1], n3, z3, eta3)


209.43951023931953
628.3185307179585
314.15926535897927
209.43951023931953


In [6]:
Etot = np.concatenate((E0, E1, E2, E3))
ztot = np.linspace(-l0, l0+l1+l2+l3, 4500)

Evstime = np.zeros((len(t), len(Etot)))
for i in range(len(t)):
    Evstime[i] = np.real(Etot * exp[i])
    

plt.plot(ztot, Evstime[0])
# plt.plot(ztot, Evstime[50])
plt.show()

In [7]:
# E0 = np.zeros(len(z0))

# for i in t:
#     E0[0] = np.sin(2*pi*f*i)
#     H0[0] = 0
#     E0 = propagation(E0)

In [8]:
fig, ax = plt.subplots()
ax.set_xlabel("z [m]")
ax.set_ylabel("Electric Field [V/m]")
ax.set_title("Field in layered media")
ax.grid()
ax.set_ylim(-3, 3)

line, = ax.plot(ztot, Evstime[0], label="Non dispersive medium")
ax.legend()

def anim(i):       
    line.set_ydata(Evstime[i])
    return line,

ani = animation.FuncAnimation(fig, anim, frames=len(t),interval=50, repeat=True ,blit=False)
plt.show()

# writer = animation.PillowWriter(fps=15, metadata=dict(artist='Me'),bitrate=1800)
# ani.save('layered.gif', writer=writer)